This implementation 

1. Reads the entity word and context word stored in doc_entity_context_word.p dictionary

Dictionary format

Element of Dictionary [ doc_id: dictonary of roles]

Key: Document names present in the corpus
Values: Dictionary of roles present in specific directory

Elements of Role Dictionary: 
Keys: Roles present in a specific doc
Value: List of entities having that role

Entity list itself is a list of words


2. Train the Doc2Vec model Store the trained model
3. Generate Doc2Vec representation for Entities at Sentence Level
4. Generate Doc2Vec representation for Entities at Doc Level

Dictionary format

Element of Dictionary [ doc_id: dictonary of roles]

Key: Document names present in the corpus
Values: Dictionary of roles present in specific directory

Elements of Role Dictionary: 
Keys: Roles present in a specific doc
Value: List of entities having that role

Entity in the Entity list is 300-d vector which is doc2vec

In [1]:
import os, pickle
import gensim, logging
from gensim.models import Word2Vec, Phrases, phrases, KeyedVectors, doc2vec
import scipy, numpy


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [13]:
data_output_folder = '../../Data/output'
train_entity_rep_output_folder = 'EntityRep/train'
test_entity_rep_output_folder = 'EntityRep/test'
entity_context_word_file = 'doc_entity_context_word.p'
entity_doc_level_context_word_file = 'doc_entity_doc_level_context_word.p'

doc_level = True
sent_level = False

word_emnbedding_pretrained_trained_on_corpus = '../../trained_word_embeddings/word2vec/word_pretrain_trained_on_corpus/w2v_pretain_corpus_trained_gensim_300.txt'

In [14]:
# Load the pickle file

if sent_level:
    train_doc_role_entity_context_word_dict = pickle.load(open(os.path.join(data_output_folder, train_entity_rep_output_folder, entity_context_word_file), 'rb'))
    test_doc_role_entity_context_word_dict = pickle.load(open(os.path.join(data_output_folder, test_entity_rep_output_folder, entity_context_word_file), 'rb'))
    
if doc_level:
    train_doc_role_entity_context_word_dict = pickle.load(open(os.path.join(data_output_folder, train_entity_rep_output_folder, entity_doc_level_context_word_file), 'rb'))
    test_doc_role_entity_context_word_dict = pickle.load(open(os.path.join(data_output_folder, test_entity_rep_output_folder, entity_doc_level_context_word_file), 'rb'))

In [4]:
word_vectors = KeyedVectors.load_word2vec_format(word_emnbedding_pretrained_trained_on_corpus, binary=False)

2018-05-24 15:44:52,196 : INFO : loading projection weights from ../../trained_word_embeddings/word2vec/word_pretrain_trained_on_corpus/w2v_pretain_corpus_trained_gensim_300.txt
2018-05-24 15:46:54,499 : INFO : loaded (403308, 300) matrix from ../../trained_word_embeddings/word2vec/word_pretrain_trained_on_corpus/w2v_pretain_corpus_trained_gensim_300.txt


In [5]:
def read_entities_build_corpus(doc_role_entity_context_word_dict, tokens_only=False):
    entities_corpus = list()
    
    for doc in doc_role_entity_context_word_dict.keys():
       
        tag_dict = doc_role_entity_context_word_dict[doc]
        
        for tag in tag_dict.keys():
            entity_list = tag_dict[tag]
            for entity in entity_list:
                if tokens_only:
                    yield(entity[1])
                else:
                    yield doc2vec.TaggedDocument(entity[1], [doc+'_'+entity[0]])
                    
                

In [15]:
train_corpus = list(read_entities_build_corpus(train_doc_role_entity_context_word_dict))
train_corpus[:2]

[TaggedDocument(words=['delay', 'reconstruction', 'blast', 'israeli', 'embassy', 'car', 'bombing', 'case', 'attributed', 'primarily', 'excessive', 'use', 'water', 'extinguish', 'fire', 'fact', 'splinters', 'used', 'bomb', 'suggests', 'specific-target', 'oriented', 'attack', 'aim', 'target', 'israeli', 'embassy', 'vehicle', 'passersby', 'impact', 'localised', 'despite', 'incident', 'taking', 'place', 'open', 'several'], tags=['2012_2_19_st-673.txt_israeli_embassy']),
 TaggedDocument(words=['delay', 'reconstruction', 'blast', 'israeli', 'embassy', 'car', 'bombing', 'case', 'attributed', 'primarily', 'excessive', 'use', 'water', 'extinguish', 'fire', 'fact', 'splinters', 'used', 'bomb', 'suggests', 'specific-target', 'oriented', 'attack', 'aim', 'target', 'israeli', 'embassy', 'vehicle', 'passersby', 'impact', 'localised', 'despite', 'incident', 'taking', 'place', 'open', 'several'], tags=['2012_2_19_st-673.txt_israeli_embassy'])]

In [16]:
test_corpus = list(read_entities_build_corpus(test_doc_role_entity_context_word_dict, tokens_only=True))
print(test_corpus[:2])

[['alert', 'agents', 'two', 'navy', 'operatives', 'one', 'coast', 'guard', 'man', 'arrived', 'varkala', 'coast', 'seagoing', 'vessel', 'shallow', 'draft', 'enabled', 'operate', 'waters', 'close', 'shore'], ['pretext', 'one', 'ill', 'fishermen', 'dropped', 'agents', 'remote', 'stretch', 'coast', 'near', 'chilakoor', 'around', '5.30', 'a.m.', 'agents', 'donned', 'type', 'dress', 'commonly', 'worn', 'white-collar']]


#### Train the model with pretrained word2vec

In [17]:
model = doc2vec.Doc2Vec(vector_size=300, min_count=1, epochs=100, pretrained_emb=word_emnbedding_pretrained_trained_on_corpus)
model.build_vocab(train_corpus)
#training_examples_count = model.corpus_count
#model.build_vocab([list(word_vectors.vocab.keys())], update=True)

#model.intersect_word2vec_format(word_emnbedding_pretrained_trained_on_corpus,binary=False, lockf=1.0)

2018-05-24 15:50:50,275 : INFO : collecting all words and their counts
2018-05-24 15:50:50,277 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-05-24 15:50:50,479 : INFO : PROGRESS: at example #10000, processed 761497 words (3797548/s), 12014 word types, 7132 tags
2018-05-24 15:50:50,588 : INFO : collected 16037 word types and 12685 unique tags from a corpus of 17806 examples and 1132747 words
2018-05-24 15:50:50,589 : INFO : Loading a fresh vocabulary
2018-05-24 15:50:50,635 : INFO : min_count=1 retains 16037 unique words (100% of original 16037, drops 0)
2018-05-24 15:50:50,636 : INFO : min_count=1 leaves 1132747 word corpus (100% of original 1132747, drops 0)
2018-05-24 15:50:50,743 : INFO : deleting the raw counts dictionary of 16037 items
2018-05-24 15:50:50,744 : INFO : sample=0.001 downsamples 35 most-common words
2018-05-24 15:50:50,745 : INFO : downsampling leaves estimated 1045387 word corpus (92.3% of prior 1132747)
2018-05-24 15:50:50,875

In [18]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

2018-05-24 15:51:01,909 : INFO : training model with 3 workers on 16037 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2018-05-24 15:51:02,932 : INFO : EPOCH 1 - PROGRESS: at 18.06% examples, 398364 words/s, in_qsize 5, out_qsize 0
2018-05-24 15:51:03,958 : INFO : EPOCH 1 - PROGRESS: at 63.60% examples, 376163 words/s, in_qsize 5, out_qsize 0
2018-05-24 15:51:04,693 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 15:51:04,742 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-24 15:51:04,744 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-24 15:51:04,745 : INFO : EPOCH - 1 : training on 1132747 raw words (1063119 effective words) took 2.8s, 376847 effective words/s
2018-05-24 15:51:05,755 : INFO : EPOCH 2 - PROGRESS: at 15.71% examples, 381073 words/s, in_qsize 5, out_qsize 0
2018-05-24 15:51:06,768 : INFO : EPOCH 2 - PROGRESS: at 56.46% examples, 351198 words/s, in_qsize 5, 

2018-05-24 15:51:38,374 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 15:51:38,438 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-24 15:51:38,468 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-24 15:51:38,469 : INFO : EPOCH - 13 : training on 1132747 raw words (1063017 effective words) took 2.9s, 369234 effective words/s
2018-05-24 15:51:39,482 : INFO : EPOCH 14 - PROGRESS: at 14.60% examples, 353406 words/s, in_qsize 6, out_qsize 0
2018-05-24 15:51:40,508 : INFO : EPOCH 14 - PROGRESS: at 48.74% examples, 319808 words/s, in_qsize 5, out_qsize 0
2018-05-24 15:51:41,612 : INFO : EPOCH 14 - PROGRESS: at 87.78% examples, 302475 words/s, in_qsize 5, out_qsize 0
2018-05-24 15:51:41,874 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 15:51:41,902 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-24 15:51:41,943 : INFO : worker thread finished; awaiting

2018-05-24 15:52:12,257 : INFO : EPOCH 26 - PROGRESS: at 28.31% examples, 466874 words/s, in_qsize 5, out_qsize 0
2018-05-24 15:52:13,291 : INFO : EPOCH 26 - PROGRESS: at 78.12% examples, 427651 words/s, in_qsize 6, out_qsize 0
2018-05-24 15:52:13,682 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 15:52:13,725 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-24 15:52:13,728 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-24 15:52:13,729 : INFO : EPOCH - 26 : training on 1132747 raw words (1063162 effective words) took 2.5s, 429339 effective words/s
2018-05-24 15:52:14,750 : INFO : EPOCH 27 - PROGRESS: at 28.31% examples, 462434 words/s, in_qsize 5, out_qsize 0
2018-05-24 15:52:15,752 : INFO : EPOCH 27 - PROGRESS: at 78.12% examples, 432350 words/s, in_qsize 5, out_qsize 0
2018-05-24 15:52:16,172 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 15:52:16,198 : INFO : worker th

2018-05-24 15:52:45,211 : INFO : EPOCH 39 - PROGRESS: at 25.38% examples, 449230 words/s, in_qsize 6, out_qsize 0
2018-05-24 15:52:46,220 : INFO : EPOCH 39 - PROGRESS: at 73.23% examples, 414482 words/s, in_qsize 5, out_qsize 0
2018-05-24 15:52:46,721 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 15:52:46,765 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-24 15:52:46,768 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-24 15:52:46,768 : INFO : EPOCH - 39 : training on 1132747 raw words (1063002 effective words) took 2.6s, 415382 effective words/s
2018-05-24 15:52:47,792 : INFO : EPOCH 40 - PROGRESS: at 19.52% examples, 404472 words/s, in_qsize 5, out_qsize 0
2018-05-24 15:52:48,797 : INFO : EPOCH 40 - PROGRESS: at 58.95% examples, 359612 words/s, in_qsize 6, out_qsize 0
2018-05-24 15:52:49,789 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 15:52:49,821 : INFO : EPOCH 40 

2018-05-24 15:53:21,465 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 15:53:21,499 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-24 15:53:21,517 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-24 15:53:21,518 : INFO : EPOCH - 50 : training on 1132747 raw words (1063285 effective words) took 2.5s, 427138 effective words/s
2018-05-24 15:53:22,559 : INFO : EPOCH 51 - PROGRESS: at 26.87% examples, 444612 words/s, in_qsize 5, out_qsize 0
2018-05-24 15:53:23,573 : INFO : EPOCH 51 - PROGRESS: at 76.76% examples, 421276 words/s, in_qsize 5, out_qsize 0
2018-05-24 15:53:24,011 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 15:53:24,025 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-24 15:53:24,065 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-24 15:53:24,066 : INFO : EPOCH - 51 : training on 1132747 raw words (1063349 eff

2018-05-24 15:53:53,440 : INFO : EPOCH 63 - PROGRESS: at 78.20% examples, 425545 words/s, in_qsize 5, out_qsize 0
2018-05-24 15:53:53,832 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 15:53:53,846 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-24 15:53:53,881 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-24 15:53:53,882 : INFO : EPOCH - 63 : training on 1132747 raw words (1063495 effective words) took 2.5s, 426777 effective words/s
2018-05-24 15:53:54,904 : INFO : EPOCH 64 - PROGRESS: at 28.24% examples, 461155 words/s, in_qsize 5, out_qsize 0
2018-05-24 15:53:55,906 : INFO : EPOCH 64 - PROGRESS: at 79.52% examples, 436740 words/s, in_qsize 5, out_qsize 0
2018-05-24 15:53:56,316 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 15:53:56,326 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-24 15:53:56,355 : INFO : worker thread finished; awaiting

2018-05-24 15:54:24,838 : INFO : EPOCH 76 - PROGRESS: at 26.87% examples, 456422 words/s, in_qsize 6, out_qsize 0
2018-05-24 15:54:25,841 : INFO : EPOCH 76 - PROGRESS: at 76.98% examples, 428932 words/s, in_qsize 5, out_qsize 0
2018-05-24 15:54:26,261 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 15:54:26,301 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-24 15:54:26,310 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-24 15:54:26,311 : INFO : EPOCH - 76 : training on 1132747 raw words (1063015 effective words) took 2.5s, 428667 effective words/s
2018-05-24 15:54:27,327 : INFO : EPOCH 77 - PROGRESS: at 28.31% examples, 464723 words/s, in_qsize 5, out_qsize 0
2018-05-24 15:54:28,350 : INFO : EPOCH 77 - PROGRESS: at 79.52% examples, 433768 words/s, in_qsize 5, out_qsize 0
2018-05-24 15:54:28,722 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 15:54:28,748 : INFO : worker th

2018-05-24 15:54:57,130 : INFO : EPOCH 89 - PROGRESS: at 28.31% examples, 465732 words/s, in_qsize 5, out_qsize 0
2018-05-24 15:54:58,134 : INFO : EPOCH 89 - PROGRESS: at 78.12% examples, 433258 words/s, in_qsize 6, out_qsize 0
2018-05-24 15:54:58,544 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 15:54:58,562 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-24 15:54:58,580 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-24 15:54:58,581 : INFO : EPOCH - 89 : training on 1132747 raw words (1062974 effective words) took 2.5s, 432285 effective words/s
2018-05-24 15:54:59,612 : INFO : EPOCH 90 - PROGRESS: at 28.31% examples, 457974 words/s, in_qsize 5, out_qsize 0
2018-05-24 15:55:00,631 : INFO : EPOCH 90 - PROGRESS: at 79.52% examples, 431510 words/s, in_qsize 6, out_qsize 0
2018-05-24 15:55:01,022 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 15:55:01,024 : INFO : worker th

CPU times: user 10min 31s, sys: 16.4 s, total: 10min 47s
Wall time: 4min 24s


In [10]:
def build_doc_role_entity_doc2vec_dictionary(doc_role_entity_context_word_dict):
    
    doc_role_entity_context_word_doc2vec_dict = dict()
    
    for doc in doc_role_entity_context_word_dict.keys():
        #print(doc)
        tag_dict = doc_role_entity_context_word_dict[doc]
        tag_doc2vec_dict = dict()
        for tag in tag_dict.keys():
            entity_list = tag_dict[tag]
            entity_doc2vec_list = list()
            for entity in entity_list:
                entity_doc2vec_list.append((entity[0], model.infer_vector(entity[1])))
            tag_doc2vec_dict[tag] = entity_doc2vec_list
        doc_role_entity_context_word_doc2vec_dict[doc] = tag_doc2vec_dict
        
    return doc_role_entity_context_word_doc2vec_dict

In [19]:
#train_doc_role_entity_context_word_centroid_dict = build_doc_role_entity_centroid_dictionary(train_doc_role_entity_context_word_dict)
test_doc_role_entity_context_word_doc2vec_dict = build_doc_role_entity_doc2vec_dictionary(test_doc_role_entity_context_word_dict)

In [20]:
if sent_level:
    pickle.dump(test_doc_role_entity_context_word_doc2vec_dict, open('../../Data/output/EntityRep/test/doc_role_entity_context_word_doc2vec_dict.p', 'wb'))

if doc_level:
    pickle.dump(test_doc_role_entity_context_word_doc2vec_dict, open('../../Data/output/EntityRep/test/doc_role_entity_doc_level_context_word_doc2vec_dict.p', 'wb'))